# ROS сервисы

На данный момент было рассмотрено много нового и предлагается немного порассуждать о том, что имеется на данный момент, а также рассмотреть еще один метод коммуникации между узлами - сервисы.

В помощь подойдут ссылки о [сервисах](http://wiki.ros.org/Services) и о том, как их писать на [Python](http://wiki.ros.org/ROS/Tutorials/WritingServiceClient%28python%29) и [C++](http://wiki.ros.org/ROS/Tutorials/WritingServiceClient%28c%2B%2B%29). Также отсюда мы научимся [создавать свои типы сервисов](http://wiki.ros.org/ROS/Tutorials/CreatingMsgAndSrv).

## Топики и сообщения

- Сообщения являются форматом передаваемых через топик данных. С ними вроде все просто =)
- Топики являются каналами передачи данных, в которые один узел кладет данные, а один или несколько узлов данные получают.
- Вряд ли существует возможность публикации в один и тот же топик из двух узлов. Вот несколько приемников - состояние нормальное.
- Через топики сообщения проходят с какой-то частотой или вразброс. Также, топик является односторонним каналом передачи. Так два узла могут организовать двухстороннюю передачу (туда-сюда) только с помощью двух топиков.

## Сервисы

Чаще всего топики используются для потоковой передачи данных, один узел просто публикует данные и те, кто на топик подписан, данные получают. Данный подход к коммуникации хорош для постоянной передачи данных, но что если нам нужно один раз передать сообщение/запрос и, возможно, получить ответ на этот запрос. Примером может выступать запрос на включение/отключение передачи данных, получение состояния в определенный момент и т.д.

Еще немного терминологии: подход с топиками можно назвать публикация-подписка, в свою очередь подход с использованием сервисов - клиент-сервер. Еще хочется заметить, что узел может сразу быть и публикатором, подписчиком, сервером, клиентом. То есть подходы коммуникации не исключают друг друга.

Так что же такое сервисы в ROS? Это запросы к ядру с получением ответа. И тут есть одна особенность, если ROS содержит огромное количество стандартных сообщений, то стандартных сервисов раз, два и обчелся =)

Так что если самодельные сообщения пишутся редко из-за обилия стандартных возможностей, то сервисы пишутся частенько и разобрать как это делается нужно. Да и по аналогии с сообщениями это уже будет не трудно =)

На этом предисловие предлагаю закончить и перейти к практике!

Для начала нам надо познакомиться с базовыми утилитами относительно сервисов: `rossrv` и `roservice`. Первая работает по аналогии с `rosmsg`, у нее даже хелп похожий

In [ ]:
%%bash
rossrv -h

Таким образом мы можем посмотреть на различные типы сервисов в ROS пакетах и системе в целом

In [ ]:
%%bash
rossrv list

Так вот как и сказано ранее - стандартных типов сервисов мало

In [ ]:
%%bash
rossrv package std_srvs

Рассмотрим из чего обычно состоит формат (тип) сервиса. Для примера возьмем сервис `rospy_tutorials/AddTwoInts`

In [ ]:
%%bash
rossrv show rospy_tutorials/AddTwoInts

Если определение сообщения включает в себя просто перечисление данных, которое несет сообщение, то для сервиса знаком '---' разделяется область запроса и область ответа (request/response). В рассмотренном сервисе два поля a и b являются данными для запроса, поле sum является данными для ответа.

То есть узел, который будет выступать в качестве сервера для сервиса данного типа будет принимать на вход два числа, а выдавать одно. По секрету скажу, что в пакете для обучения сервис складывает два числа, только тсссс =)

Что же про `rosservice`, он уже ближе к утилите `rostopic`.
> Для дальнейшей работы стоит стартануть мастера и узел `add_two_ints_server` из пакета `rospy_tutorials`

Запустили? Поехали проверять:

In [ ]:
%%bash
rosservice -h

In [ ]:
%%bash
rosservice list

In [ ]:
%%bash
rosservice info /add_two_ints

In [ ]:
%%bash
rosservice args /add_two_ints

А теперь очень хитрый ход конем, вызовем сервис из терминала и проверим результат

In [ ]:
%%bash
rosservice call /add_two_ints 1 2

И что мы видим? Вызов подставил числа в аргументы a и b и произвел запрос к узлу-серверу сервиса типа `rospy_tutorials/AddTwoInts`. Вот так мы познакомились с базовыми утилитами для работы с сервисами! На этом можно узел-сервер закрыть.

## Создание типа сервиса

А давайте сделаем свой сервис с блэк-джеком и, кхм, полиномами? =)

Определяем сервис, создавая папку в нашем пакете под названием `srv` и внутри файл `Poly.srv`. Принцип тот же - как файл назовешь, так сервис и поплывет.
Содержание:
```
int64 x
---
int64 y
```

Как вы правильно догадались - надо бы дополнить CMakeLists.txt. Открываем =)  

Такие строки мы уже добавляли, но ведь мы пишем полное руководство, так что для генерации типа сервиса это надо
```cmake
find_package(
    catkin REQUIRED COMPONENTS 
    ... 
    std_msgs 
    message_generation 
)
```

Также аналогично добавляем макросы
```cmake
catkin_package( 
    ... 
    CATKIN_DEPENDS message_runtime ... 
    ...
)
```

А вот теперь добавляем файлы типов сервисов к генерации
```cmake
add_service_files(
    FILES
    ...
    Poly.srv
)
```

И снова повторяет этап, который является общим как для сообщений, так и для типов сервисов
```cmake
generate_messages( 
    DEPENDENCIES 
    ...
    std_msgs
)
```

Сохраняем и открываем файл package.xml, в который мы уже добавляли эти строки, но опять же - руководство полное, так что упомянуть надо бы
```xml
  <build_depend>message_generation</build_depend>
  <exec_depend>message_runtime</exec_depend>
```

После нехитрых махинаций (по факту рядом с сообщениями добавили один макрос) вызываем `catkin_make` внутри ws.

Следующая ячейка проверит, что в пакете присутствует новый тип сервиса (если название пакета другое - измените немного ячейку)

In [ ]:
%%bash
rossrv package study_pkg

## Сервис Python

Новый тип определили, пора бы и узел-сервер написать, который будет обрабатывать запросы данного типа сервиса. Полный код имеет следующий вид
```python
#!/usr/bin/env python

from study_pkg.srv import Poly, PolyResponse
import rospy

def handle_poly_srv(req):
    result = req.x + req.x ** 2
    print "Returning [%s + %s^2 = %s]" % (req.x, req.x, result)
    
    return PolyResponse(y=result)

def poly_server():
    rospy.init_node('poly_server')
    s = rospy.Service('poly', Poly, handle_poly_srv)
    print "Ready to calc polynomial."
    rospy.spin()

poly_server()
```

Начнем с импортов, импортируем общий `rospy` и модуль типа сервиса `Poly` вместе с типом ответа `PolyResponse`
```python
from study_pkg.srv import PolyResponse, Poly
import rospy
```

Вообще при генерации сервиса создается три типа: `Poly`, `PolyRequest`, `PolyResponse`. В данном случае нам интересен только сам тип сервиса и ответ.

Далее пишем функцию-обработчик, которая будет обрабатывать приходящие запросы. Аргумент обычно имеет тип `PolyRequest`, поэтому мы можем обращаться к полям запроса напрямую (х)
```python
def handle_poly_srv(req):
    result = req.x + req.x ** 2
    print "Returning [%s + %s^2 = %s]" % (req.x, req.x, result)
    
    return PolyResponse(y=result)
```

В самой функции вычисляется требуемый полином и возврат проиводится объектом `PolyResponse`, в конструкторе которого сразу заполнили единственное поле (y). Также есть вариант сделать возврат следующим образом (эквивалентно первому, но длинее)
```python
def handle_poly_srv(req):
    result = req.x + req.x ** 2
    print "Returning [%s + %s^2 = %s]" % (req.x, req.x, result)
    
    resp = PolyResponse()
    resp.y = result
    
    return resp
```

После этого пишем основную инициализацию, в которой регистрируем узел и сервис. При регистрации сервиса передаем имя сервиса (которое кстати тоже можно мапировать =)), тип и функцию обработчик. После чего запускаем этот сервер (так как писали внутри функции `poly_server()` - вызываем ее)
```python
def poly_server():
    rospy.init_node('poly_server')
    s = rospy.Service('poly', Poly, handle_poly_srv)
    print "Ready to calc polynomial."
    rospy.spin()

poly_server()
```

> Создайте в папке `scripts` файл poly_service.py и перенесите полный код в него. Не забудьте дать права на выполнение: `chmod +x poly_service.py`. Проверьте работоспособность сервиса через утилиты терминала.

## Клиент сервиса Python

Так как мы убедились, что наш сервис работает - попробуем вызвать его программными средствами из другого узла. Данный пример будет сделан в ячейках блокнота, так как тут нет никаких бесконечных циклов.

> Для дальнейшей работы должен быть запущен сервис poly_service.py, который мы только написали.

Начнем, импортируем необходимы модули

In [ ]:
import rospy
from study_pkg.srv import Poly, PolyRequest, PolyResponse

Далее вызываем функцию ожидания регистрации сервиса, ведь может так получиться, что клиент запустили, а сервиса еще нет =(  
Для этого есть специальная функция `wait_for_service()`

In [ ]:
rospy.wait_for_service('poly')

Далее получаем объект сервиса функцией `rospy.ServiceProxy()` и через него вызываем сервис с заполненным объектом `PolyRequest` и получаем объект `PolyResponse`. В случае каких-либо проблем при передаче будет вызвано исключение, которое обработается конструкцией `try-catch`

In [ ]:
try:
    poly_srv = rospy.ServiceProxy('poly', Poly)
    req = PolyRequest(x=5)
    # resp - объект типа PolyResponse
    resp = poly_srv(req)
    print( resp.y )
except rospy.ServiceException, e:
    print("Service call failed: %s" % e)

Вывод ячейки и терминал сервиса показывают, что все работает корректно.

> Попробуйте создать узел в пакете и повызывать его через `rosrun`. Таким образом в пакете будут лежать два узла: клиент и сервер сервиса.

## Сервисы С++

На этом предлагаю остановиться и немного порассуждать в сторону. С++ сервисы абсолютно ничем не отличаются кроме синтаксиса языка. С одной стороны предлагается обратиться к [офф странице по написанию клиента и сервера сервиса на С++](http://wiki.ros.org/ROS/Tutorials/WritingServiceClient%28c%2B%2B%29). Так как ранее уже была рассмотрена самая тяжелая часть по поводу компиляции, в данном блоке просто рассмотрим основные отличные от Python моменты и в качестве задания будет написать узлы клиента и сервера с суффиксами `_cpp` на С++.

Начнем конечно с заголовков
```cpp
#include "study_pkg/Poly.h"
```

Далее функция обработки запросов, здесь типизация строгая, поэтому все жестко
```cpp
bool poly_cb(study_pkg::Poly::Request  &req,
             study_pkg::Poly::Response &res)
{
    res.y = req.x + req.x * req.x;
    ROS_INFO("request: x=%ld", (long int)req.x);
    ROS_INFO("sending back response: [%ld]", (long int)res.y);
    return true;
}
```

При регистрации сервиса пользуемся своим типом
```cpp
ros::ServiceServer service = n.advertiseService("poly", poly_cb);
```

На этом основные каверзные моменты относительно сервера сервиса рассмотрены. Перейдем к клиенту, создание объекта клиента сервиса
```cpp
ros::ServiceClient client = n.serviceClient<study_pkg::Poly>("poly");
```

Создание объекта запроса сервиса и вызов
```cpp
study_pkg::Poly srv;
srv.request.x = 5;
if (client.call(srv))
{
    ROS_INFO("Poly: %ld", (long int)srv.response.y);
}
else
{
    ROS_ERROR("Failed to call service add_two_ints");
    return 1;
}
```

> Создать узлы клиента и сервера созданного сервиса на языке С++ и проверить их работоспособность.

> Попробуйте вызвать C++ клиент при запущенном Python сервере сервиса и наоборот. Убедитесь в отсутствии привязки к языку.

## В результате

- Рассмотрели еще один метод коммуникации - сервисы.
- Научились создавать собственные типы сервисов.
- Рассмотрели создание клиентов и серверов сервисов на языках Python и C++.